# Merchant Rating and Ranking

In [28]:
import pandas as pd
import numpy as np

In [11]:
import ray

ray.shutdown()

In [ ]:
ray.init()

In [2]:
all_order_train = pd.read_pickle('../all_order_train.pkl')
print(all_order_train.shape)

all_order_train.head()

(33720820, 12)


,order_id,user_id,order_number,order_hour_of_day,product_id,purchase_date,merchant,product_name,price,aisle,department,week_number
0,2539329,1,1,8,196,2019-02-13,L&M Markets (Hometown Grocers Co-op),Soda,3.08,soft drinks,beverages,7
1,2231262,31,17,11,196,2019-03-10,L&M Markets (Hometown Grocers Co-op),Soda,3.08,soft drinks,beverages,10
2,3058369,195,34,10,196,2019-02-25,L&M Markets (Hometown Grocers Co-op),Soda,3.08,soft drinks,beverages,9
3,2257155,951,3,20,196,2019-03-09,L&M Markets (Hometown Grocers Co-op),Soda,3.08,soft drinks,beverages,10
4,1121647,992,7,10,196,2019-03-08,L&M Markets (Hometown Grocers Co-op),Soda,3.08,soft drinks,beverages,10


## Calculate Rating Score

* `customer_loyalty = returned customers this period / total customers this period`, [0,1] range
* `sales_performance = # of orders above past average sales this period/ # of orders this period`, [0,1] range
* `rating = 1 - sqrt((power(1-customer_loyalty, 2) + (power(1-sales_performance, 2))/2)`
  * It uses the distance between perfect score 1 and current point

In [45]:
@ray.remote
def get_customer_loyalty(current_df, current_week, merchant):
    past_clients = set(current_df.loc[current_df['week_number'] < current_week]['user_id'].values)
    current_clients = set(current_df.loc[current_df['week_number'] == current_week]['user_id'].values)
    returned_clients = past_clients.intersection(current_clients)
    customer_loyalty = len(returned_clients)/len(current_clients)
    
    return {'merchant': merchant, 'week_number': current_week, 'customer_loyalty': round(customer_loyalty, 4)}


@ray.remote
def get_sales_performance(current_df, current_week, merchant):
    past_avg_sales = np.mean(current_df.loc[current_df['week_number'] < current_week]['price'].values)
    current_sales = current_df.loc[current_df['week_number'] == current_week]['price'].values
    above_avg_sales = [v for v in current_sales if v >= past_avg_sales]
    sales_performance = len(above_avg_sales)/len(current_sales)
    
    return {'merchant': merchant, 'week_number': current_week, 'sales_performance': round(sales_performance, 4)}

In [40]:
merchant_lst = all_order_train['merchant'].unique()
week_lst = all_order_train['week_number'].unique()
week_lst.sort()

customer_loyalty_results = [get_customer_loyalty.remote(all_order_train.loc[(all_order_train['merchant']==merchant) 
                                         & (all_order_train['week_number'] <= current_week)][['user_id', 'week_number']],
                                                       current_week, merchant)
                           for merchant in merchant_lst
                           for current_week in week_lst[1:]]
customer_loyalty_lst = ray.get(customer_loyalty_results)

In [53]:
sales_performance_results = [get_sales_performance.remote(all_order_train.loc[(all_order_train['merchant']==merchant) 
                                         & (all_order_train['week_number'] <= current_week)][['price', 'week_number']],
                                                       current_week, merchant)
                           for merchant in merchant_lst
                           for current_week in week_lst[1:]]
sales_performance_lst = ray.get(sales_performance_results)

In [54]:
customer_loyalty_df = pd.DataFrame(customer_loyalty_lst)
sales_performance_df = pd.DataFrame(sales_performance_lst)

In [43]:
customer_loyalty_df.head()

,merchant,week_number,customer_loyalty
0,L&M Markets (Hometown Grocers Co-op),8,0.3653
1,L&M Markets (Hometown Grocers Co-op),9,0.7708
2,L&M Markets (Hometown Grocers Co-op),10,0.8876
3,L&M Markets (Hometown Grocers Co-op),11,0.9171
4,L&M Markets (Hometown Grocers Co-op),12,0.9551


In [55]:
sales_performance_df = pd.DataFrame(sales_performance_lst)
sales_performance_df.head()

,merchant,week_number,sales_performance
0,L&M Markets (Hometown Grocers Co-op),8,0.4921
1,L&M Markets (Hometown Grocers Co-op),9,0.3243
2,L&M Markets (Hometown Grocers Co-op),10,0.3107
3,L&M Markets (Hometown Grocers Co-op),11,0.3093
4,L&M Markets (Hometown Grocers Co-op),12,0.3392
